In [1]:
from textwrap import wrap, indent, shorten
from faker import Faker
from uniseg.linebreak import line_break_boundaries
from uniseg.wordbreak import words
from hyphen import Hyphenator

Faker.seed(0xdeadbeef)
lorem = Faker(["la"]).paragraph(62)

print("Generated lorem:\n")
print("\n".join(indent(ln, "    ") for ln in wrap(lorem)))
print("")

DISCRETIONARY_HYPHEN = "\u00AD"
hyphenator = Hyphenator()
hyphenated_words = []
for word in words(lorem):
    syllables = hyphenator.syllables(word)
    if "".join(syllables) == word:
        hyphenated_words.extend(f"{s}{DISCRETIONARY_HYPHEN}" for s in syllables[:-1])
        hyphenated_words.extend(syllables[-1:])
    else:
        hyphenated_words.append(word)

lorem = "".join(hyphenated_words)

# Generate a list of feasible indices where lines could start
feasible_line_start_indices = [0]
feasible_line_start_indices.extend(line_break_boundaries(lorem))
feasible_line_start_indices.pop()

print("Final few few feasible line starting points:\n")
for idx in feasible_line_start_indices[-10:]:
    print(f"    {shorten(lorem[idx:], 30)!r}")

Generated lorem:

    Iusto nobis incidunt nostrum quas occaecati. Ex facilis pariatur quam
    reprehenderit fugit magni. Eius dolorum dolorum maxime hic. Illum nemo
    neque consectetur voluptatum perspiciatis ab. Optio molestias nobis
    error facere libero. Harum ipsam natus reprehenderit dolore assumenda
    unde. Tenetur ab corrupti rem modi quia fuga. Aliquam pariatur dolores
    esse consectetur ipsa fugit. Beatae perferendis alias debitis. Eius
    minus voluptas amet quis quae. Fuga corrupti laudantium incidunt in
    ipsa tempora. Unde veritatis quia iure voluptatem fugit culpa. Minima
    excepturi saepe vero sit officia. At eum reprehenderit veritatis quos.
    Neque possimus molestiae veritatis molestias autem fugiat minima.
    Laudantium sapiente alias illo totam. Officia sed soluta harum
    incidunt. Nemo voluptatibus magnam ducimus labore officiis. Earum modi
    sunt explicabo totam ab. Sequi laborum molestias sit a laudantium.
    Quas distinctio ullam voluptatem

In [2]:
import math
from wcwidth import wcswidth
from typing import Optional
from functools import lru_cache

OVERFULL_BADNESS = 1000000
MAX_BADNESS = 10000

def render(line: str, width: int, *, justify=False) -> str:
    # Strip any trailing whitespace
    ends_with_soft_hyphen = line.endswith(DISCRETIONARY_HYPHEN)
    line = line.rstrip()
    line = line.replace(DISCRETIONARY_HYPHEN, "")

    if not justify:
        return line
    
    words_in_line = list(words(line))
    natural_width = wcswidth(line)
    assert natural_width >= 0
    if ends_with_soft_hyphen:
        natural_width += 1
    
    n_whitespace = sum(1 for w in words_in_line if w.isspace())
    whitespace_len = 1.0
    if n_whitespace > 0:
        whitespace_len += (width - natural_width) / n_whitespace

    components = []
    ws_error = 0.
    for word in words_in_line:
        components.append(word)
        if word.isspace():
            ws_error += whitespace_len - 1.
            while ws_error >= 0.5:
                components.append(" ")
                ws_error -= 1.

    if ends_with_soft_hyphen:
        components.append("-")
        
    return "".join(components)

def line_badness(line: str, ideal_width: int, *, max_width: Optional[int] = None) -> int:
    """
    Compute 'badness' of rendering a given line into a particular
    ideal width and maximum width.
    """
    # Max width defaults to ideal width
    if max_width is None:
        max_width = ideal_width

    # What is the natural width?
    ends_with_soft_hyphen = line.endswith(DISCRETIONARY_HYPHEN)
    line = line.rstrip()
    line = line.replace(DISCRETIONARY_HYPHEN, "")
    natural_width = wcswidth(line)
    if ends_with_soft_hyphen:
        natural_width += 1
    if natural_width < 0:
        raise ValueError(f"Input line has non-printable characters: {line!r}")

    # Compute the badness score for the line.
    if max_width < natural_width:
        return OVERFULL_BADNESS
    return min(MAX_BADNESS, (ideal_width - natural_width) ** 2)

In [6]:
from sortedcontainers import SortedSet

# List of current "best" spltting for each element in feasible_line_start_indices.
# The list is either "None" if there is no current best splitting or a
# (badness, remaining_starts) pair where remaining_starts is a list of indices into
# feasible_line_start_indices for the remaining line breaks in this solution.
best_solutions: list[Optional[tuple[int, list[int]]]] = [None] * len(feasible_line_start_indices)

# List of indices into feasible_line_start_indices representing endpoints of
# lines we want to explore
to_search: SortedSet[int] = SortedSet(key=lambda idx: best_solutions[idx][0])

# Wrapping parameters.
ideal_width = 50
max_width = ideal_width

# Badness added when a line is broken.
BREAK_PENALTY = 10

# Badness added when breaking on a soft hyphen.
SOFT_HYPHEN_PENALTY = 50

# Initialise best solutions table with "trivial" last lines.
for idx in range(0, len(feasible_line_start_indices)):
    feasible_line_start_idx = len(feasible_line_start_indices) - 1 - idx
    line_start_idx = feasible_line_start_indices[feasible_line_start_idx]
    line = lorem[line_start_idx:]
    badness = line_badness(line, ideal_width, max_width=max_width)
    # We always _try_ to grow the paragraph, even if that means having maximal badness.
    if badness >= MAX_BADNESS and idx > 0:
        break
    best_solutions[feasible_line_start_idx] = (badness, [])
    to_search.add(feasible_line_start_idx)

# We consider each new line starting point given an ending point. Because
# we keep the end point queue sorted by badness, we encourage searching
# for optimal solutions.
iter_count = 0
while len(to_search) > 0:
    iter_count += 1
    feasible_line_end_idx = to_search.pop(0)
    line_end_idx = feasible_line_start_indices[feasible_line_end_idx]
    prev_badness, prev_line_starts = best_solutions[feasible_line_end_idx]
    remaining_line_starts = [feasible_line_end_idx]
    remaining_line_starts.extend(prev_line_starts)
    
    for feasible_line_start_idx in range(feasible_line_end_idx-1, -1, -1):
        line_start_idx = feasible_line_start_indices[feasible_line_start_idx]
        line = lorem[line_start_idx:line_end_idx]            
        badness = line_badness(line, ideal_width, max_width=max_width)

        # We always _try_ to grow the paragraph, even if that means having maximal badness.
        if badness >= MAX_BADNESS and feasible_line_start_idx != feasible_line_end_idx - 1:
            break
            
        badness += prev_badness
        if line.endswith(DISCRETIONARY_HYPHEN):
            badness += SOFT_HYPHEN_PENALTY

        if feasible_line_start_idx != 0:
            badness += BREAK_PENALTY

        best_solution_entry = best_solutions[feasible_line_start_idx]
        if best_solution_entry is None or best_solution_entry[0] > badness:
            best_solutions[feasible_line_start_idx] = (badness, remaining_line_starts)
            to_search.add(feasible_line_start_idx)

print(f"Feasible line break location count: {len(feasible_line_start_indices)}")
print(f"Iteration count: {iter_count}, Lowest badness: {best_solutions[0][0]}")

Feasible line break location count: 400
Iteration count: 492, Lowest badness: 773


In [7]:
optimal_lines = []
line_ends = [feasible_line_start_indices[idx] for idx in best_solutions[0][1]]
line_ends.append(len(lorem))
line_start = 0
for line_end in line_ends:
    optimal_lines.append(lorem[line_start:line_end])
    line_start = line_end

greedy_lines = []
current_line = []
start_idx = 0
for end_idx in feasible_line_start_indices + [len(lorem)]:
    block = lorem[start_idx:end_idx]
    line_len = wcswidth(f"{''.join(current_line)}{block.rstrip()}".replace(DISCRETIONARY_HYPHEN, ""))
    assert line_len >= 0
    if block.endswith(DISCRETIONARY_HYPHEN):
        line_len += 1

    if line_len > max_width:
        greedy_lines.append("".join(current_line))
        current_line = [block]
    else:
        current_line.append(block)
    start_idx = end_idx
if len(current_line) > 0:
    greedy_lines.append("".join(current_line))

justify = True

def pad(s: str, width: int):
    sw = wcswidth(s)
    assert sw >= 0
    padding = max(0, width - sw)
    return f"{s}{' ' * padding}"

print()
print(f"    {pad('Greedy', max_width)}    |    {pad('Optimal', max_width)}")
ruler = f"{'.' * ideal_width}{'+' * (max_width - ideal_width)}"
print(f"    {pad(ruler, max_width)}    |    {pad(ruler, max_width)}")
for idx in range(max(len(greedy_lines), len(optimal_lines))):
    try:
        greedy_line = greedy_lines[idx]
    except IndexError:
        greedy_line = ""

    try:
        optimal_line = optimal_lines[idx]
    except IndexError:
        optimal_line = ""

    g = render(greedy_line, ideal_width, justify=idx != len(greedy_lines) - 1)
    o = render(optimal_line, ideal_width, justify=idx != len(optimal_lines) - 1)
    print(f"    {pad(g, max_width)}    |    {pad(o, max_width)}")
print(f"    {pad(ruler, max_width)}    |    {pad(ruler, max_width)}")


    Greedy                                                |    Optimal                                           
    ..................................................    |    ..................................................
    Iusto  nobis incidunt  nostrum quas  occaecati. Ex    |    Iusto  nobis incidunt  nostrum quas  occaecati. Ex
    facilis pariatur  quam reprehenderit  fugit magni.    |    facilis pariatur  quam reprehenderit  fugit magni.
    Eius dolorum dolorum maxime  hic. Illum nemo neque    |    Eius  dolorum  dolorum   maxime  hic.  Illum  nemo
    consectetur voluptatum perspiciatis  ab. Optio mo-    |    neque  consectetur   voluptatum  perspiciatis  ab.
    lestias nobis error facere libero. Harum ipsam na-    |    Optio molestias  nobis error facere  libero. Harum
    tus reprehenderit  dolore assumenda  unde. Tenetur    |    ipsam natus  reprehenderit dolore  assumenda unde.
    ab corrupti  rem modi quia fuga.  Aliquam pariatur    |    Tenetur ab  corrupti rem